In [47]:
import pandas as pd
import numpy
pd.options.mode.chained_assignment = None

# Sentiment analysis

- Preprocessing
    - remove URLs
    - remove Stopwords
    - Lemmatization
- Sentiment Analysis with nltk

# Crawling Tweets with Hashtag '?'

In [2]:
#pip install tweepy

In [3]:
import os
import tweepy as tw

In [4]:
consumer_key= 'vvoCCekgIRltBaA28KPeqV5PH'
consumer_secret= 'wbJdfJgPQo6bojNVWtX2FoH8RNXMG7OA7xNis1HOS5tt4E3kN7'
access_token= '216684460-uXpFa0MbWSjZyKrLLajc42VELc7GzFZPIs53PV3g'
access_token_secret= 'quTUm4pTmVv9Uy2AZanWQcErohcDdoFWzUWg17pLtTriC'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [14]:
# Define the search term and the date_since date as variables
search_words = "bitcoin"
date_since = "2022-01-22"

In [17]:
tweets = tw.Cursor(api.search_tweets, 
                           q=search_words,
                           lang="en", #since=date_since
                  ).items(2000)

users_tweets = [[tweet.id, tweet.user.screen_name, tweet.text, tweet.created_at, tweet.user.location] for tweet in tweets]

Rate limit reached. Sleeping for: 55


In [18]:
#Open dataframe to use Sentiment Analysis
tweet_text = pd.DataFrame(data=users_tweets, 
                    columns=['id', 'user', "text", "created_at", "location"])
tweet_text.head()

,id,user,text,created_at,location
0,1486094472798629888,BrizendineSean,RT @Cointelegraph: The dip isn’t over just yet...,2022-01-25 21:51:47+00:00,"Santa Rosa, California"
1,1486094471980802059,yashi_jr,RT @__Cryptopy__: @TheMoonCarl I'm buying the ...,2022-01-25 21:51:47+00:00,
2,1486094470906978305,jillpembrook,@devrima69799930 @NFTONETHWeb3 @MetaGiveaway T...,2022-01-25 21:51:46+00:00,United States
3,1486094470575792128,Btc_Kgee,I might marry a white girl cause atleast they ...,2022-01-25 21:51:46+00:00,South Africa
4,1486094466557657088,StandardFine,RT @NickAllen: Bitcoin down to $2.70 a pop. So...,2022-01-25 21:51:45+00:00,Milano


---

In [22]:
#pip install nltk

In [23]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

# NLP
## Preprocessing

In [24]:
dataset = tweet_text

In [29]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /Users/femu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/femu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/femu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/femu/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

Set lowercase, remove punctuation, remove url:
- "no","nor","not" removed from stopwords because they may be relevant for sentiment

In [48]:
#def clean_dataset(dataset):
for i in range(0, len(dataset)):
    #Tokenize and set words to lowercase 
    review = dataset["text"][i]
    review = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",review).split())
    review = re.sub("[^a-zA-Z]", " ", review)
    review = review.lower()
    review = review.split()

    #stopwords: 
    all_stopwords = [word for word in stopwords.words("english") if word not in ["no","nor","not"]]
    all_stopwords.extend(["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","&amp","amp","amp;","&amp;","nhs", "rt"])
    #lemmatization:
    lemma = nltk.wordnet.WordNetLemmatizer()
    review = " ".join([lemma.lemmatize(word) for word in review if word not in set(all_stopwords)])    
    dataset["text"][i] = review


In [36]:
dataset.head()

,id,user,text,created_at,location
0,1486094472798629888,BrizendineSean,dip yet say bitcoin derivative data show trade...,2022-01-25 21:51:47+00:00,"Santa Rosa, California"
1,1486094471980802059,yashi_jr,cryptopy buying dip bitcoin breakout bullishaf...,2022-01-25 21:51:47+00:00,
2,1486094470906978305,jillpembrook,trading bitcoin real first thought wa,2022-01-25 21:51:46+00:00,United States
3,1486094470575792128,Btc_Kgee,might marry white girl cause atleast believe b...,2022-01-25 21:51:46+00:00,South Africa
4,1486094466557657088,StandardFine,bitcoin pop glad buy mess,2022-01-25 21:51:45+00:00,Milano


## Sentiment Analysis with nltk

In [37]:
tweet_text["sentiment"] = dict
tweet_text["sentiment_compound"] = 0.0

Determine sentiment via NLTK.Sentiment:
- Values range from [-1, 1]
- -1 is negative, 0 is neutral, 1 is positive

In [49]:
def sentiment_score(dataset):
    sia = SentimentIntensityAnalyzer()
    for i in range(len(dataset)):
        dataset["sentiment"][i] = sia.polarity_scores(dataset["text"][i])
        dataset["sentiment_compound"][i] = dataset["sentiment"][i]["compound"]

In [50]:
sentiment_score(tweet_text)

In [51]:
tweet_text.head()

,id,user,text,created_at,location,sentiment,sentiment_compound
0,1486094472798629888,BrizendineSean,dip yet say bitcoin derivative data show trade...,2022-01-25 21:51:47+00:00,"Santa Rosa, California","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
1,1486094471980802059,yashi_jr,cryptopy buying dip bitcoin breakout bullishaf...,2022-01-25 21:51:47+00:00,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
2,1486094470906978305,jillpembrook,trading bitcoin real first thought wa,2022-01-25 21:51:46+00:00,United States,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
3,1486094470575792128,Btc_Kgee,might marry white girl cause atleast believe b...,2022-01-25 21:51:46+00:00,South Africa,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
4,1486094466557657088,StandardFine,bitcoin pop glad buy mess,2022-01-25 21:51:45+00:00,Milano,"{'neg': 0.294, 'neu': 0.353, 'pos': 0.353, 'co...",0.128


In [52]:
tweet_ids = tweet_text[['id', 'sentiment', 'sentiment_compound']]

In [53]:
tweet_ids.head()

,id,sentiment,sentiment_compound
0,1486094472798629888,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
1,1486094471980802059,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
2,1486094470906978305,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
3,1486094470575792128,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000
4,1486094466557657088,"{'neg': 0.294, 'neu': 0.353, 'pos': 0.353, 'co...",0.128


Sort dataset by sentiment and create a sub-dataset with location only:

In [54]:
tweet_ids = tweet_text.sort_values("sentiment_compound", ascending=False, ignore_index=True)

---

Export:

In [55]:
tweet_ids.to_csv("../sentiment/saved_tweet_ids/tweet_ids.csv")
tweet_text.to_csv("saved_sentiment_data/tweet_complete.csv")